In [1]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../fuzzy-torch') # or just install the module
sys.path.append('../../fuzzy-tools') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [2]:
from fuzzytools.files import search_for_filedirs
from lchandler import _C as _C

surveys_rootdir = '../../surveys-save/'
filedirs = search_for_filedirs(surveys_rootdir, fext=_C.EXT_SPLIT_LIGHTCURVE)

────────────────────────────────────────────────────────────────────────────────────────────────────
found filedirs: (../../surveys-save/)
(0) - ../../surveys-save//survey=alerceZTFv7.1~bands=gr~mode=onlySNe.splcds - 16.973[mbs]
(1) - ../../surveys-save//survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method=spm-mcmc-estw.splcds - 225.085[mbs]
(2) - ../../surveys-save/wiib/survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method=spm-mcmc-fstw.splcds - 226.801[mbs]
(3) - ../../surveys-save/wiib/survey=alerceZTFv7.1~bands=gr~mode=onlySNe.splcds - 17.073[mbs]
(4) - ../../surveys-save/wiib/survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method=bspline-fstw.splcds - 226.809[mbs]
(5) - ../../surveys-save/wiib/survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method=spm-mcmc-estw.splcds - 226.818[mbs]
(6) - ../../surveys-save/wiib/survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method=linear-fstw.splcds - 226.784[mbs]
────────────────────────────────────────────────────────────────────────────────────────────────────
fil

In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from fuzzytools.files import load_pickle, save_pickle
from fuzzytools.files import get_dict_from_filedir

method = 'spm-mcmc-estw'
filedir = f'../../surveys-save/survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method={method}.splcds'
kf = '3'

filedict = get_dict_from_filedir(filedir)
root_folder = filedict['_rootdir']
cfilename = filedict['_cfilename']
survey = filedict['survey']
lcdataset = load_pickle(filedir)
lcdataset.only_keep_kf(kf) # saves ram
print(lcdataset)

LCDataset:
[3@train; samples 1,185]
(.) obs_samples=32,461; min_len=6; max_dur=538.8[days]; dur(p50)=52.8[days]; cadence(p50)=1.0[days]
(g) obs_samples=14,376; min_len=0; tmax=10.99609375; max_dur=538.8 [days]; dur(p50)=39.0 [days]; cadence(p50)=3.0 [days]
(r) obs_samples=18,085; min_len=0; tmax=13.03515625; max_dur=538.7 [days]; dur(p50)=50.0 [days]; cadence(p50)=3.0 [days]
   |        | SLSN - 18/1,185 (1.52%)
   |█▍      | SNII* - 208/1,185 (17.55%)
   |██████  | SNIa - 899/1,185 (75.86%)
   |▍       | SNIbc - 60/1,185 (5.06%)
────────────────────────────────────────────────────────────────────────────────────────────────────
[3@val; samples 395]
(.) obs_samples=11,003; min_len=6; max_dur=443.9[days]; dur(p50)=53.0[days]; cadence(p50)=1.1[days]
(g) obs_samples=4,813; min_len=0; tmax=11.01953125; max_dur=443.9 [days]; dur(p50)=37.0 [days]; cadence(p50)=3.0 [days]
(r) obs_samples=6,190; min_len=0; tmax=13.01953125; max_dur=443.9 [days]; dur(p50)=52.0 [days]; cadence(p50)=3.0 [days]
  

In [4]:
lcset = lcdataset[f'{kf}@train']
values = lcset.get_all_values('obs')
print(np.min(values), np.percentile(values, 50), np.max(values))
print(np.sort(values)[::-1][:100])

0.008175607 0.0950818657875061 4.7439365
[4.7439365 4.728768  4.6024437 4.5164213 4.487468  4.1022983 4.0470786
 3.769974  3.6512647 3.6495333 3.6165085 3.3707736 3.3525372 3.2578235
 3.257067  3.1890793 3.1319213 3.1071947 3.0659616 3.065003  3.0458355
 2.9732525 2.95734   2.9562514 2.9120116 2.8981762 2.8925304 2.8760905
 2.8747578 2.8605256 2.8314006 2.8096883 2.7891712 2.7854521 2.7763386
 2.7760704 2.7667835 2.742585  2.7388897 2.7351944 2.6897085 2.6475594
 2.6399443 2.632687  2.6166713 2.5760531 2.5428755 2.5371962 2.510414
 2.5090759 2.4998431 2.4929862 2.4685748 2.4585624 2.387114  2.386603
 2.3837314 2.3812578 2.3770962 2.3756201 2.358135  2.3553631 2.339419
 2.3246276 2.3090413 2.3074012 2.2865891 2.2806764 2.2761836 2.2743788
 2.2704368 2.2700996 2.2457845 2.2425735 2.2410789 2.2372339 2.230249
 2.2219906 2.221613  2.2149978 2.2099335 2.1769078 2.1455574 2.140894
 2.134225  2.1149647 2.1110241 2.1055741 2.0834832 2.0693638 2.065581
 2.0467865 2.0431678 2.039133  2.0324624 2

In [5]:
lcset = lcdataset[f'{kf}@train.{method}']
values = lcset.get_all_values('obs')
print(np.min(values), np.percentile(values, 50), np.max(values))
print(np.sort(values)[::-1][:100])

0.008175607 0.0999055951833725 5.3186426
[5.3186426 5.31499   5.3128767 5.3088145 5.2958484 5.2914796 5.2879496
 5.279645  5.259554  5.2595024 5.2577667 5.256598  5.25193   5.2507725
 5.2458935 5.2416883 5.241085  5.240665  5.239827  5.2387986 5.2381587
 5.236587  5.2358522 5.2320266 5.2284856 5.224207  5.2234483 5.2224436
 5.212849  5.2081075 5.207992  5.207511  5.2073374 5.2072067 5.207117
 5.205889  5.205211  5.205022  5.2037244 5.2030544 5.202432  5.2018414
 5.1985393 5.1960855 5.1931286 5.19279   5.191692  5.190909  5.1895723
 5.189486  5.188065  5.1877046 5.1847925 5.184452  5.182047  5.1801596
 5.178171  5.177001  5.1768975 5.176535  5.174741  5.174621  5.172786
 5.171776  5.1699605 5.167176  5.1670346 5.163765  5.163728  5.1614914
 5.1614695 5.160258  5.1600246 5.1598434 5.1589694 5.1562347 5.153978
 5.152119  5.1512594 5.1486216 5.1483574 5.1469207 5.146613  5.1422496
 5.1381025 5.1360183 5.1329246 5.132838  5.132612  5.132346  5.1323175
 5.1319447 5.131526  5.1281443 5.127006

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from lchandler.plots.distrs import plot_values_distribution

kf = 0
set_name = f'{kf}@train'
lcdataset[set_name].set_diff_parallel('days')
plot_values_distribution(lcdataset, set_name, 'd_days')
plot_values_distribution(lcdataset, set_name, 'obs')
plot_values_distribution(lcdataset, set_name, 'obse')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KeyError: '0@train'

In [7]:
%load_ext autoreload
%autoreload 2
from lcclassifier.datasets import CustomDataset

dataset_kwargs = {
    'max_day':100.,
    #attrs':['days','obs', 'obse']
    'in_attrs':['obs', 'obse'],
    #'attrs':['d_days','obs', 'obse']
    'rec_attr':'obs',
}
repeats = 5
device = 'cpu'
#lcset_name = f'{kf}@train.{method}'
lcset_name = f'{kf}@train'
s_train_dataset_da = CustomDataset(lcset_name, copy(lcdataset[lcset_name]), device,
    balanced_repeats=repeats,
    precomputed_copies=2, # 1 8 16
    uses_daugm=True,
    uses_dynamic_balance=True,
    ds_mode={'random':.75, 'left':.0, 'none':.25,},
    **dataset_kwargs,
    )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'torch'

In [ ]:
%load_ext autoreload
%autoreload 2
s_train_dataset_da.calcule_precomputed(verbose=1, n_jobs=1)

In [ ]:
assert 0

In [ ]:
assert 0

In [ ]:
%load_ext autoreload
%autoreload 2
from lcclassifier.datasets import CustomDataset


lcset_name = f'{main_args.kf}@train.{main_args.method}'
s_train_dataset_da = CustomDataset(lcset_name, copy(lcdataset[lcset_name]), device,
    balanced_repeats=repeats,
    precomputed_copies=8, # 1 8 16
    uses_daugm=True,
    uses_dynamic_balance=True,
    ds_mode={'random':.75, 'left':.0, 'none':.25,},
    **dataset_kwargs,
    )

device = 'cpu' # cpu
train_dataset = CustomDataset(f'{kf}@train.{method}', copy(lcdataset[f'{kf}@train.{method}']), device, **dataset_kwargs)
val_dataset = CustomDataset(f'{kf}@val', copy(lcdataset[f'{kf}@val']), device, **dataset_kwargs)
train_dataset.transfer_scalers_to(val_dataset) # transfer metadata to val/test
print('train_dataset:', train_dataset)
print('val_dataset:', val_dataset)

In [ ]:
%load_ext autoreload
%autoreload 2
from fuzzytorch.utils import print_tdict
import cProfile

p = cProfile.Profile(); p.enable()
tdict = train_dataset.get_item(train_dataset.get_lcobj_names()[0])
print_tdict(tdict)
p.disable(); p.dump_stats('prof.prof')
print(tdict['target']['balanced_w'])
pass

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from lchandler.plots.lc import plot_lightcurve
from lchandler import C_ as C_
%matplotlib inline

dataset = train_dataset
lcobj_name = dataset.get_random_stratified_lcobj_names()[0]


tdict, lcobj = dataset.get_item(lcobj_name, uses_len_clip=False, uses_daugm=False, return_lcobjs=True)
print(lcobj)


minput = tdict['input']
target = tdict['target']

figsize = (10,13)
fig, axs = plt.subplots(5+1, 1, figsize=figsize)

ax = axs[0]
for kb,b in enumerate(dataset.band_names):
    plot_lightcurve(ax, lcobj, b, label=f'{b} obs', max_day=dataset.max_day)
ax.set_ylabel('observation')

b = 'r'
len_lcobj = minput[f'onehot.{b}'].sum()
ax = axs[1]
time = minput[f'rtime.{b}'][...,0]
for ka,in_attr in enumerate(dataset.in_attrs):
    ax.plot(time[:len_lcobj], minput[f'x.{b}'][:len_lcobj,ka], '-o', label=f'{C_.SHORT_NAME_DICT[in_attr]} (norm)')
ax.set_ylabel(f'x.{b}')

ax = axs[2]
ax.plot(time[:len_lcobj], minput[f'onehot.{b}'][:len_lcobj], 'o')
ax.set_ylabel(f'onehot.{b}')

ax = axs[3]
ax.plot(time[:len_lcobj], minput[f'rtime.{b}'][:len_lcobj], '-o')
ax.set_ylabel(f'time.{b}')

ax = axs[4]
ax.plot(time[:len_lcobj], minput[f'dtime.{b}'][:len_lcobj], '-o')
ax.set_ylabel(f'dtime.{b}')

ax = axs[5]
ax.plot(time[:len_lcobj], target[f'rec_x.{b}'][:len_lcobj], '-o')
ax.set_ylabel(f'rec_x.{b}')

class_name = dataset.class_names[target['y']]
title = ''
title += f'training light curve sample & model inputs & onehot & temporal encoding \n'
title += f'survey: {dataset.lcset.survey} - set: {dataset.lcset_name}'
title += f' - class: {class_name} - max_day: {dataset.max_day:.2f} - max_len: {dataset.max_len}'
#title += f' - training: {dataset.training}'
for ax in axs:
    #ax.legend(prop={'size':14})
    ax.legend(loc='upper right')
    ax.grid(alpha=0.5)
axs[0].set_title(title)
axs[-1].set_xlabel('days')
plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2
from lcclassifier.dataloaders import CustomDataLoader
from fuzzytorch.utils import print_tdict

loader_kwargs = {
    'batch_size':2,
}
random_subcrops = 3
s_train_loader = CustomDataLoader(train_dataset, shuffle=False, **loader_kwargs)
s_train_loader.eval()
dataset.set_max_day(40)

for k,tdict in enumerate(s_train_loader):
    target = tdict['target']
    print_tdict(tdict)
    print(tdict['input']['rtime.*'][0,:,0])
    break

In [ ]:
%load_ext autoreload
%autoreload 2
from lcclassifier.dataloaders import CustomDataLoader
from fuzzytorch.utils import print_tdict

loader_kwargs = {
    'batch_size':1,
    #'num_workers':1, # bug?
}
random_subcrops = 3
s_train_loader = CustomDataLoader(train_dataset, shuffle=True, random_subcrops=random_subcrops, **loader_kwargs)
s_train_loader.train()

for k,tdict in enumerate(s_train_loader):
    model_input = tdict['input']
    target = tdict['target']
    print_tdict(tdict)
    for idx in range(len(model_input['x'])):
        print(model_input['x'][idx,:,0])
        print(model_input['onehot'][idx].sum(-1))
    assert 0